In [2]:
import math
import pandas as pd
import numpy as np

In [3]:
df = pd.DataFrame(
    {
        'sku': [100, 200, 300, 400, 500, ],
        'gmv':   [400, 350, 500, 800, 0, ], 
        'price': [100, 70,  120, 200, 50],
        'stock': [3,   10,  5,  0, 100],

        
        
        # 'gmv':   [290, 300, 400, 350, 300, 0,  4, 0, 230], 
        # 'price': [100, 100, 100, 70,  120, 10, 4, 0, 230],
        # 'stock': [3,   3,   3,   10,  5,   1,  0, 0, 0],

    }
)
df

,sku,gmv,price,stock
0,100,400,100,3
1,200,350,70,10
2,300,500,120,5
3,400,800,200,0
4,500,0,50,100


In [4]:
# corner cases - zero floor mixed?

def update_gmv(gmv, price, stock):
    """."""
    if stock == 0:
        return 0

    if price == 0:
        return 0
    
    if gmv == 0:
        return 0
    
    if (stock < 0) or (price < 0) or (gmv < 0):
        return 0
    
    # if gmv > price * stock:
    #     return price * stock
    
    if math.floor(gmv / price) > stock:
        return stock  * price
    
    if gmv % price != 0:
        return math.floor(gmv / price) * price

    return gmv


def limit_gmv(df: pd.DataFrame) -> pd.DataFrame:
    """."""
    # 1 version
    gmv_upd = df.apply(lambda x: update_gmv(x['gmv'], x['price'], x['stock']), axis=1)
    df2 = pd.DataFrame()
    df2['sku'] = df['sku']
    df2['gmv'] = gmv_upd
    df2[['price', 'stock']] = df[['price', 'stock']]
    df2['stock'] = df['stock']
    
    # 2 version
    # gmv_upd = df.apply(lambda x: update_gmv(x['gmv'], x['price'], x['stock']), axis=1)
    # df2 = df.copy()
    # df2.drop(columns=['gmv'], axis=1, inplace=True)
    # df2['gmv'] = gmv_upd
    # columns_order = ['sku', 'gmv', 'price', 'stock']
    # df2 = df2[columns_order]
    
    return df2

%timeit df2 = limit_gmv(df)

2.32 ms ± 285 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [138]:
df2

,sku,gmv,price,stock
0,100,300,100,3
1,200,350,70,10
2,300,480,120,5
3,400,0,200,0
4,500,0,50,100


In [164]:
df_arr = np.array(df)
df_arr

array([[100, 400, 100,   3],
       [200, 350,  70,  10],
       [300, 500, 120,   5],
       [400, 800, 200,   0],
       [500,   0,  50, 100]])

In [229]:
arr = df_arr[2]
# np.int32(np.floor(arr[1] / arr[2]))
arr

array([300, 500, 120,   5])

In [237]:
arr[1] % arr[2]

20

In [238]:
500 % 120

20

In [262]:
df

,sku,gmv,price,stock
0,100,400,100,3
1,200,350,70,10
2,300,500,120,5
3,400,800,200,0
4,500,0,50,100


In [264]:
arr[1] <= 0

False

In [5]:
def vect_gmv(arr):
    if arr[1] <= 0: return 0
    if arr[2] < 0: return 0
    if arr[3] <= 0: return 0

    if arr[1] > (arr[2] * arr[3]):
        return arr[2] * arr[3]

    # temp = np.int32(np.floor(arr[1] / arr[2]))

    if arr[1] % arr[2] != 0:
        return math.floor(arr[1] / arr[2]) * arr[2]
    
    return arr[1]


def limit_gmv(df: pd.DataFrame) -> pd.DataFrame:
    """."""
    df_arr = np.array(df)
    gmv_upd = np.apply_along_axis(func1d=vect_gmv, axis=1, arr=df_arr)

    return pd.DataFrame(
        {
            'sku': df_arr[:, 0], 
            'gmv': gmv_upd, 
            'price': df_arr[:, 2],
            'stock': df_arr[:, 3],
        }
    )

# %timeit  
# np.apply_along_axis(func1d=vect_gmv, axis=1, arr=df_arr)

# %timeit df2 = limit_gmv(df)
df2 = limit_gmv(df)
df2

,sku,gmv,price,stock
0,100,300,100,3
1,200,350,70,10
2,300,480,120,5
3,400,0,200,0
4,500,0,50,100


In [249]:
df_arr = np.array(df)
df_arr

array([[100, 400, 100,   3],
       [200, 350,  70,  10],
       [300, 500, 120,   5],
       [400, 800, 200,   0],
       [500,   0,  50, 100]])

In [245]:
arr[2] * arr[3]

600

In [211]:
df_arr = np.array(df)
df_arr

array([[100, 400, 100,   3],
       [200, 350,  70,  10],
       [300, 500, 120,   5],
       [400, 800, 200,   0],
       [500,   0,  50, 100]])

In [216]:
# df_arr[:, 0]

temp = pd.DataFrame(
    {
        'sku': df_arr[:, 0]
    }
)
temp



,sku
0,100
1,200
2,300
3,400
4,500


In [165]:
b = np.array([[1,2,3], [4,5,6], [7,8,9]])
pd.DataFrame(b)

,0,1,2
0,1,2,3
1,4,5,6
2,7,8,9


In [181]:
b

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [167]:
def my_func(a):
    return (a[0] + a[-1]) * 0.5
np.apply_along_axis(my_func, 0, b)

array([4., 5., 6.])

In [124]:
arr = df['sku']
arrgmv = df['gmv']

temp = pd.DataFrame()
temp['sku'] = arr
temp['gmv'] = arrgmv
temp

,sku,gmv
0,100,400
1,200,350
2,300,500
3,400,800
4,500,0


In [125]:
temp

,sku,gmv
0,100,400
1,200,350
2,300,500
3,400,800
4,500,0


In [260]:
np.where(
    (
        (df_arr[:, 1] <= 0) or (df_arr[:, 2] <= 0) or (df_arr[:, 3] <= 0)
    ), 
    0,
    df_arr[:, 3],
)


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [114]:
type(df.apply(lambda x: update_gmv(x['gmv'], x['price'], x['stock']), axis=1))

pandas.core.series.Series

In [75]:
temp = df
temp

,gmv,price,stock
0,290,100,3
1,300,100,3
2,400,100,3
3,350,70,10
4,300,120,5
5,0,10,1
6,4,4,0
7,0,0,0
8,230,230,0


In [76]:
df['temp'] = df['stock']
df

,gmv,price,stock,temp
0,290,100,3,3
1,300,100,3,3
2,400,100,3,3
3,350,70,10,10
4,300,120,5,5
5,0,10,1,1
6,4,4,0,0
7,0,0,0,0
8,230,230,0,0


In [77]:
temp

,gmv,price,stock,temp
0,290,100,3,3
1,300,100,3,3
2,400,100,3,3
3,350,70,10,10
4,300,120,5,5
5,0,10,1,1
6,4,4,0,0
7,0,0,0,0
8,230,230,0,0


In [21]:
df2 = limit_gmv(df)
df2

,price,stock,gmv_upd
0,100,3,300
1,70,10,350
2,120,5,300
3,10,1,0
4,4,0,0
5,0,0,0


In [13]:
from random import random
import numpy as np

In [43]:
df3 = pd.DataFrame(
    {
        'gmv': np.random.randint(low=-100, high=1500, size=50), 
        'price': np.random.randint(low=-100, high=1500, size=50), 
        'stock': np.random.randint(low=-100, high=1500, size=50), 
    }
)
df3

,gmv,price,stock
0,451,1437,175
1,1328,1266,966
2,932,-63,406
3,1007,442,1155
4,1497,1297,1224
5,970,1322,1332
6,858,-3,119
7,959,529,140
8,1006,543,601
9,33,716,1290


In [ ]:
df['stock_check'] = df4[]

In [45]:
df4 = limit_gmv(df3)
df4['stock_check'] = df4['gmv'] / df4['price']

df4

    # if math.floor(gmv / price) > stock:
    #     return stock * price
    # return gmv

# price	stock	gmv	stock_check
# 1437	175	451	0.313848

,price,stock,gmv,stock_check
0,1437,175,451,0.313848
1,1266,966,1328,1.048973
2,-63,406,0,-0.000000
3,442,1155,1007,2.278281
4,1297,1224,1497,1.154202
5,1322,1332,970,0.733737
6,-3,119,0,-0.000000
7,529,140,959,1.812854
8,543,601,1006,1.852670
9,716,1290,33,0.046089


In [24]:
23 | 1


23

In [25]:
True | False


True

In [26]:
True or False

True

In [101]:
float(math.floor(10.0/3.0))


3.0